<font size = "5"> 1.1 </font>
<body>
<font size = "2" color = "red">->Put raw csv in same path as ipynb for smooth running of code </font><br><br>

<font size = "3">This part of the code finds the number of houses in to whole dataset,<br>
outputs <b><i>clean_df</i></b> csv for further processing,<br>
marks out the time period of all malfunction meter</font><br>
</body>

In [76]:
import numpy as np
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
import timeit
from datetime import timedelta
from pathlib import Path
# start_time = timeit.default_timer() * 1000
currentPath = str(Path().resolve())
# Put raw csv in same 
path = currentPath + "./dataport-export_gas_oct2015-mar2016.csv"

# Read in raw data and find 
df = pd.read_csv(path)
data_id = df['dataid'].nunique()
print("Number of houses =", data_id)

FileNotFoundError: File b'C:\\Users\\G552\\Desktop\\4211 proj./dataport-export_gas_oct2015-mar2016.csv' does not exist

In [74]:
# Sort data into order by id, if same id then by time
df.sort_values(["dataid", "localminute"], ascending=[True, True], inplace = True)
df.reset_index(drop = True, inplace = True)

# Cast localminute into proper time format for further processing
df.localminute = df.localminute.str.slice(0,19)
df.localminute = pd.to_datetime(df.localminute, infer_datetime_format = True, format = "%Y/%m/%d %I:%M:%S %p")
print(df.localminute[0])
print("Length of df:", len(df))

2015-10-01 00:14:44
Length of df: 1584823


,localminute,dataid,meter_value
1583860,2015-12-11 03:17:43,9982,109704
1583861,2015-12-11 04:24:58,9982,109704
1583862,2015-12-11 05:23:50,9982,109704
1583863,2015-12-11 05:48:51,9982,109704
1583864,2015-12-11 07:05:40,9982,109706
1583865,2015-12-11 07:29:42,9982,109706
1583866,2015-12-11 08:21:54,9982,109708
1583867,2015-12-11 08:44:59,9982,109716
1583868,2015-12-11 10:12:41,9982,109720
1583869,2015-12-11 11:52:48,9982,109724


In [75]:
# Types of malfunction
# 1: data reported back when change in gas use is < 2 cubic foot
# 2: data reported back new meter_value is smaller than old meter_value
# 3: data reported back is >2, but time > 15s, threshold value for reporting malfunction will be 7 cubic foot/hr 
# (US household average daily usage = 168 cubic foot) https://www.aga.org/sites/default/files/aga_2961_2015_aga_playbook_final_0.pdf
# If report between 2 reading is > 168, the extra high reading will be treated as malfunction and dropped away
# 4: However, continuous report of same reading from same id over 12hrs,
# the first reading after 12hrs will be treated as good reading,
# as the gas company will probably want to know whether is a meter malfunctioning
# even though its reading did not change for a period of time

# Progress bar cuz I always feel it is not working
# f = FloatProgress(min=0, max=(len(df['dataid']) - 1))
# display(f)

# Create empty df and array for more efficient removal of item in df
malfunction = pd.DataFrame(columns = ["localminute", "dataid", "meter_value"])
bad_array = []
to_drop = []

prev_good = True
_id = None
_12hr = None
# Sort malfunction with time period label
# Assumption: the first data point is always correct as the 2nd pt is wrt to it, 3rd wrt to 2nd....etc
# Append malfunction period and data to respective array
for row in df.itertuples():
    if(_id is None or _id != row.dataid):
        prev_good = True
        _id = row.dataid
    # for estimating time take to process actual data
    if (row.Index == 0):
        continue
    #1 and #2
    if (((row.meter_value <= df.meter_value[row.Index-1])
         and (row.dataid == df.dataid[row.Index-1])) 
        or ((row.localminute != df.localminute[row.Index-1]) 
            and (row.dataid == df.dataid[row.Index-1]) 
            and ((row.meter_value - df.meter_value[row.Index-1]) < 2))):
        if(prev_good == True):
            bad_array.append([_id, row.localminute, row.localminute])
            to_drop.append(row.Index)
            #4, update 12hr pt
            _12hr = row.localminute
        else:
            #4  
            if ((row.localminute - _12hr) >= pd.to_timedelta("12:00:00")):
                prev_good = True
                continue
            else:                
                if ((df.meter_value[row.Index - 1] - row.meter_value) < 168):
                    to_drop.append(row.Index)
                    bad_array[-1][2] = row.localminute
                    
        prev_good = False
    else:
        #3
        if ((row.meter_value - df.meter_value[row.Index - 1]) < 168):
            if(((row.meter_value - df.meter_value[row.Index-1]) > 2) 
               and ((((row.localminute - df.localminute[row.Index-1]) / timedelta(hours = 1)) * 7) 
                    > (row.meter_value - df.meter_value[row.Index - 1]))):
                bad_array[-1][2] = row.localminute
            prev_good = True
        else:
            to_drop.append(row.Index)
            prev_good = False
            bad_array[-1][2] = row.localminute
            
print("Length of bad_array:", len(bad_array))
print("Length of to_drop:", len(to_drop))

109704 1st bad
109704 bad
109704 bad
109706 good
109706 1st bad
109708 good
109716 good
109720 <168 but rate too big
109720 good
109724 <168 but rate too big
109724 good
123708 >168
109726 good..?
109734 <168 but rate too big
109734 good
109734 1st bad
109734 bad
109734 bad
109736 good
109738 good
109746 <168 but rate too big
109746 good
109746 1st bad
Length of bad_array: 4
Length of to_drop: 9


,localminute,dataid,meter_value
1583860,2015-12-11 03:17:43,9982,109704
1583864,2015-12-11 07:05:40,9982,109706
1583866,2015-12-11 08:21:54,9982,109708
1583867,2015-12-11 08:44:59,9982,109716
1583868,2015-12-11 10:12:41,9982,109720
1583869,2015-12-11 11:52:48,9982,109724
1583871,2015-12-11 12:29:49,9982,109726
1583872,2015-12-11 14:10:52,9982,109734
1583876,2015-12-11 20:17:46,9982,109736
1583877,2015-12-12 02:59:43,9982,109738


In [33]:
#Remove malfunction data to produce clean df
df.drop(index = to_drop, inplace = True)
df.reset_index(drop = True, inplace = True)

# # Cast to int64
# malfunction['dataid'] = malfunction['dataid'].astype(np.int64)
# malfunction['meter_value'] = malfunction['meter_value'].astype(np.int64)

# # Merge 2 df to compare diff, if one of the label value is different,
# # value in _merge label will be different hence being able to differeniate the difference
# df = pd.merge(df, malfunction, on=['localminute', 'dataid', 'meter_value'], how='outer', indicator=True)\
# .query("_merge != 'both'")\
# .drop(['_merge'], axis=1)\
# .reset_index(drop=True)

# Convert bad_array into df and transform it for shape to be correct
malfunction = pd.DataFrame(bad_array, columns = ['dataid', 'start_time', 'end_time'])
malfunction.T

# Save part1 result into csv for easier access in part2
df.to_csv('./clean_df.csv', index = False)
malfunction.to_csv('./malfunction.csv', index = False)

# pd.set_option('display.max_rows', 10000)

# elapsed = timeit.default_timer() * 1000 - start_time
# print("total: %ds" %(elapsed/1000)) if ((elapsed > 5000) == True) else print("total: %dms" %elapsed)
print("length of df:", len(df))
print("length of malfunction:", len(malfunction))

length of df: 53
length of malfunction: 21
